## Tabela Silver para Dimenssão de Calendario

#### Tratamento do Dataframe para obter informações desejadas

Aqui vamos filtrar apenas as colunas desejadas e realizar o tratamentod e tipagem e limpeza de dados necessárias

Vamos realizar alguns tratamentos

    Seleção de colunas necessárias
    Configurar colunas para o padrão
    Filtrando dados indesejados
    Alteração de tipagem de dados
    Removendo dados nulos
    Salvando arquivo de forma compactada

In [0]:
# Importando Bibliotecas para uso do Spark
from pyspark.sql import SparkSession # Import para inciar o spark
from pyspark.sql.types import * # Todos as configurações do SQL de tipagem no spark
from pyspark.sql.functions import * # Todas as funções de SQL do Spark
from pyspark.sql.window import Window

In [0]:
# Diretorio Bronze, com os dados brutos das causas dos acidentes
Bronze_Acidentes_Causas = '/Volumes/workspace/dw_acidentes/01_bronze/Acidentes_Causas/'

# Local para armazenar dimenssão
Diretorio_Silver = '/Volumes/workspace/dw_acidentes/02_silver'


In [0]:
# Lê os arquivos CSV descompactado
df_Acidentes_Causas = spark.read.parquet(Bronze_Acidentes_Causas)

In [0]:
# Selecionado apenas as colunas necessária para criar o modelo Dimenssional de Clima
df_calendario = df_Acidentes_Causas.select(['data_inversa', 'dia_semana'])

In [0]:
# Removendo Duplicadas para criar o campo chave
df_calendario = df_calendario.dropDuplicates()

In [0]:
# Criando a função de janela e ordenando todos as colunas aplicando linha a linha 
window = Window.orderBy('data_inversa', 'dia_semana')

# Aplicando a função row_number e over para cada linha incluir o ID incremental.
df_calendario = df_calendario.withColumn(
    "id_calendario",
    row_number().over(window)
)

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
# Enriquecendo adicionado informações como trimestre, mês e dia
df_calendario = (
    df_calendario
    .withColumn("trimestre", quarter("data_inversa"))
    .withColumn("mes", month("data_inversa"))
    .withColumn("dia", dayofmonth("data_inversa"))
)

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
# Enriquecedo realizado calculo para se obter a semana do mês na tabela
df_calendario = df_calendario.withColumn(
    "semana_do_mes",
    floor(
        (
            dayofmonth("data_inversa")
            + dayofweek(date_trunc("month", "data_inversa"))
            - 1
        ) / 7
    ) + 1
)

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
# Enriquecedo realizado calculo para se obter a semana do ano na tabela
df_calendario = df_calendario.withColumn("semana_do_ano", weekofyear("data_inversa"))

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
# Realizado condicional para se obter o nome do mês em Portugês na tabela
df_calendario = df_calendario.withColumn(
    "nome_do_mes",
    when(month("data_inversa") == 1, "Janeiro")
    .when(month("data_inversa") == 2, "Fevereiro")
    .when(month("data_inversa") == 3, "Março")
    .when(month("data_inversa") == 4, "Abril")
    .when(month("data_inversa") == 5, "Maio")
    .when(month("data_inversa") == 6, "Junho")
    .when(month("data_inversa") == 7, "Julho")
    .when(month("data_inversa") == 8, "Agosto")
    .when(month("data_inversa") == 9, "Setembro")
    .when(month("data_inversa") == 10, "Outubro")
    .when(month("data_inversa") == 11, "Novembro")
    .otherwise("Dezembro")
)

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
# Renomeado a coluna para padrão desejado
df_calendario = df_calendario.withColumnRenamed('data_inversa', 'data_acidente')

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
# Definido a posição das colunas conforme documetnação
df_calendario = df_calendario.select(['id_calendario', 'data_acidente', 'dia_semana', 'trimestre', 'mes', 'dia', 'semana_do_mes', 'semana_do_ano', 'nome_do_mes'])

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
# Avaliando se o schema atende a documentação
df_calendario.printSchema()

root
 |-- id_calendario: integer (nullable = false)
 |-- data_acidente: date (nullable = true)
 |-- dia_semana: string (nullable = true)
 |-- trimestre: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)
 |-- semana_do_mes: long (nullable = true)
 |-- semana_do_ano: integer (nullable = true)
 |-- nome_do_mes: string (nullable = false)



In [0]:
# Avaliando os primeiros Registros
display(df_calendario.limit(5))

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


id_calendario,data_acidente,dia_semana,trimestre,mes,dia,semana_do_mes,semana_do_ano,nome_do_mes
1,2017-01-01,domingo,1,1,1,1,52,Janeiro
2,2017-01-02,segunda-feira,1,1,2,1,1,Janeiro
3,2017-01-03,terça-feira,1,1,3,1,1,Janeiro
4,2017-01-04,quarta-feira,1,1,4,1,1,Janeiro
5,2017-01-05,quinta-feira,1,1,5,1,1,Janeiro


In [0]:
# Salvando o arquivo em parquet com compressão gzip 
df_calendario.write.parquet(
    f"{Diretorio_Silver}/Dim_Calendario", 
    mode="overwrite",         # Aqui defino para sempre sobrescrever o arquivo
    compression="gzip"        # Comando para salvar o arquivo com compactação
)

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(
